In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import tqdm.auto as tqdm
import torch
import numpy as np
from torch import distributions, nn, optim

## Data Generation

In [2]:
snr_ranges = list(range(-40, 41, 10))

sigma_list = []

for snr in snr_ranges:
    sigma_sq = 10 ** (-snr / 10)
    sigma = sigma_sq**0.5
    sigma_list.append(sigma)

sigma_list

[100.0,
 31.622776601683793,
 10.0,
 3.1622776601683795,
 1.0,
 0.31622776601683794,
 0.1,
 0.03162277660168379,
 0.01]

In [3]:
torch.set_default_dtype(torch.float64)

N = 128
M = 20
K = 16

sigma = 0.1

In [4]:
H = np.random.normal(0,1, size=(N,M))
H = torch.from_numpy(H).double()

x_true = np.random.choice([-1,1], size=M)
x_true = torch.tensor(x_true).double()

noise = torch.from_numpy(np.random.normal(0, sigma, size=N))

R = H @ x_true + noise

y = (np.sign(R) + 1)/2
y = torch.tensor(y, dtype=torch.float64)


y.shape, H.shape, x_true.shape

C:\Users\Naman\AppData\Local\Temp\ipykernel_10756\3126022190.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float64)


(torch.Size([128]), torch.Size([128, 20]), torch.Size([20]))

In [5]:
x_true

tensor([ 1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,
        -1., -1.,  1., -1., -1., -1.])

In [6]:
y

tensor([1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
        0., 0.])

## Model

In [7]:
from pvi.models.logistic_regression import LogisticRegressionModel
from pvi.clients import Client
from pvi.servers.sequential_server import SequentialServer
from pvi.distributions.exponential_family_distributions import MultivariateGaussianDistribution, MeanFieldGaussianDistribution
from pvi.distributions.exponential_family_factors import MultivariateGaussianFactor, MeanFieldGaussianFactor


In [8]:
client_data = []

# Distribute data H and Y into K clients.
L = int(N / K)
for i in range(K):
    client_H = torch.tensor(H[L*i:L*i+L])
    client_y = torch.tensor(y[L*i:L*i+L])
    
    client_data.append({'x': client_H, 'y': client_y})

client_data[0]

C:\Users\Naman\AppData\Local\Temp\ipykernel_10756\2495994765.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  client_H = torch.tensor(H[L*i:L*i+L])
C:\Users\Naman\AppData\Local\Temp\ipykernel_10756\2495994765.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  client_y = torch.tensor(y[L*i:L*i+L])


{'x': tensor([[-0.4614, -0.2654, -0.6287,  0.1950,  0.0733,  1.2703, -0.7774, -0.3991,
          -1.1503,  1.1476,  0.7530,  0.9419,  0.4615, -1.5189, -0.6588, -2.0512,
           0.9266, -0.4626,  0.3328, -1.1961],
         [-0.0659, -2.5162,  1.0898, -0.1512, -0.0769,  0.9765, -0.2290,  1.9435,
           1.1739,  0.7933, -0.0586, -0.2185,  0.6972, -1.5136, -1.2258, -0.5211,
           0.4446,  0.1743, -0.2844,  1.1021],
         [ 0.5999, -0.3970,  0.8424,  0.6401, -0.9480, -1.4625,  0.1814, -0.3795,
          -1.2125, -0.4653, -2.1691, -0.3747, -0.2169,  0.0443,  0.8787, -1.0396,
          -0.8427,  1.9342, -1.0106,  1.6766],
         [ 0.4235,  2.2681,  0.9758, -0.4553, -0.2380,  0.1686,  0.3330,  1.2128,
           0.9846,  0.0404,  0.6690,  2.1655,  0.0664, -1.3107,  0.3410,  0.7671,
           0.8045,  0.9252,  0.8895,  1.5086],
         [-0.2800,  0.6324, -2.1592,  0.6225, -0.3431, -1.0162,  1.2479, -0.6872,
           0.7986, -0.9075,  0.1074, -0.3177, -0.4175,  0.0241,  1.37

In [9]:
hyperparameters = {
    "D": M,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 1e-2},
    "epochs": 100,
    "batch_size": 100,
    "num_elbo_samples": 1,
    "num_predictive_samples": 10
}

prior_std_params = {
    "loc": torch.zeros(hyperparameters["D"] + 1),
    "scale": torch.ones(hyperparameters["D"] + 1),
}

init_nat_params = {
    "np1": torch.zeros(hyperparameters["D"] + 1),
    "np2": torch.zeros(hyperparameters["D"] + 1),
}


In [10]:
clients = []

for i in range(K):
    # Defining a model, data known to it and 't' for each client
    model_i = LogisticRegressionModel(hyperparameters=hyperparameters)
    data_i = client_data[i]
    t_i = MeanFieldGaussianFactor(nat_params=init_nat_params)

    client = Client(data=data_i, model=model_i, t=t_i, config=hyperparameters)
    clients.append(client)
    
clients

In [11]:
model = LogisticRegressionModel(hyperparameters=hyperparameters)
q = MeanFieldGaussianDistribution(std_params=prior_std_params, is_trainable=False)
server = SequentialServer(model=model, p=q, clients=clients)
server.timer.start()
server._tick()

0it [00:00, ?it/s]

In [12]:
mean_x = np.array(server.q.std_params['loc'][:20])
detected_x = np.sign(list(mean_x))
x_true = np.array(x_true)

mean_x, detected_x, x_true

(array([ 0.83012035,  1.27854624, -0.59139142,  0.88043268, -0.66490992,
        -1.15209325, -0.71362409,  1.3423492 , -1.2565487 , -0.42504821,
        -1.14919432,  0.87345534,  1.34145613, -0.55153207, -0.76766096,
        -1.04205509,  1.16688615, -0.94991457, -0.4472713 , -1.01963586]),
 array([ 1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,
        -1., -1., -1.,  1., -1., -1., -1.]),
 array([ 1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,
        -1., -1., -1.,  1., -1., -1., -1.]))

: 

## Model Parameters